<a href="https://colab.research.google.com/github/siriwatsc-debug/DADS5001-HW2/blob/main/DADS5001_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Claude

In [3]:
!pip install pandas requests beautifulsoup4 plotly lxml html5lib selenium
#pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 27.8 MB/s eta 0:00:00


In [12]:
# Web scrape data

from selenium import webdriver
import pandas as pd

def web_driver():
    options = webdriver.ChromeOptions()
#    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
#    options.add_argument('--disable-gpu')
#    options.add_argument("--window-size=1920, 1200")
#    options.add_argument('--disable-dev-shm-usage')
    # options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()
data=[]
#driver.get('https://fbref.com/en/comps/9/Premier-League-Stats#all_results2025-202691')
driver.get('https://fbref.com/en/comps/9/Premier-League-Stats')
data = driver.page_source


df = pd.read_html(data, attrs={"id":"results2025-202691_overall"})
df

/tmp/ipython-input-1347495061.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(data, attrs={"id":"results2025-202691_overall"})


[    Rk            Squad  MP  W  D  L  GF  GA  GD  Pts  Pts/MP    xG   xGA  \
 0    1          Arsenal   9  7  1  1  16   3  13   22    2.44  14.6   5.2   
 1    2      Bournemouth   9  5  3  1  16  11   5   18    2.00  11.0  11.1   
 2    3        Tottenham   9  5  2  2  17   7  10   17    1.89   9.9  10.7   
 3    4       Sunderland   9  5  2  2  11   7   4   17    1.89   8.6  10.4   
 4    5  Manchester City   9  5  1  3  17   7  10   16    1.78  15.6   9.0   
 5    6   Manchester Utd   9  5  1  3  15  14   1   16    1.78  16.2  13.6   
 6    7        Liverpool   9  5  0  4  16  14   2   15    1.67  15.8  12.0   
 7    8      Aston Villa   9  4  3  2   9   8   1   15    1.67   7.1  11.8   
 8    9          Chelsea   9  4  2  3  17  11   6   14    1.56  13.4  13.1   
 9   10   Crystal Palace   9  3  4  2  12   9   3   13    1.44  17.6  11.7   
 10  11        Brentford   9  4  1  4  14  14   0   13    1.44  13.4  12.2   
 11  12    Newcastle Utd   9  3  3  3   9   8   1   12    1.33  

In [15]:
# Homework

import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import re

'''
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    return driver
'''

# Scrape data from Web
def scrape_premier_league_data(url):
    """
    Scrape Premier League statistics from FBref
    """

    try:
        time.sleep(2)
        options = webdriver.ChromeOptions()
        options.add_argument('--no-sandbox')
        options.add_argument('--headless')
        driver = webdriver.Chrome(options=options)
        data=[]

        driver.get(url)
        # Read data from webpage
        data = driver.page_source
        print(driver)

        if driver:

            df = pd.read_html(data, attrs={"id":"results2025-202691_overall"})
            print(f"✅ ดึงข้อมูลสำเร็จ! พบ {len(df)} ทีม")
            print(f"📊 Columns: {df.columns.tolist()}")
            return df

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        print("💡 ใช้ข้อมูลจากภาพที่แนบมา")

    return create_data_from_image()

def create_data_from_image():
    """
    สร้างข้อมูลจากภาพที่แนบมา (ข้อมูลจริง Premier League 2024-25)
    """
    data = {
        'Rk': list(range(1, 21)),
        'Squad': ['Arsenal', 'Bournemouth', 'Tottenham', 'Sunderland', 'Manchester City',
                 'Manchester Utd', 'Liverpool', 'Aston Villa', 'Chelsea', 'Crystal Palace',
                 'Brentford', 'Newcastle Utd', 'Brighton', 'Everton', 'Leeds United',
                 'Burnley', 'Fulham', 'Nottingham Forest', 'West Ham', 'Wolves'],
        'MP': [9] * 20,
        'W': [7, 5, 5, 5, 5, 5, 5, 4, 4, 3, 4, 3, 3, 3, 3, 3, 2, 1, 1, 0],
        'D': [1, 3, 2, 2, 1, 1, 0, 3, 2, 4, 1, 3, 3, 2, 2, 1, 2, 2, 1, 2],
        'L': [1, 1, 2, 2, 3, 3, 4, 2, 3, 2, 4, 3, 3, 4, 4, 5, 5, 6, 7, 7],
        'GF': [16, 16, 17, 11, 17, 15, 16, 9, 17, 12, 14, 9, 14, 9, 9, 12, 9, 5, 7, 7],
        'GA': [3, 11, 7, 7, 7, 14, 14, 8, 11, 9, 14, 8, 15, 12, 14, 17, 14, 17, 20, 19],
        'GD': [13, 5, 10, 4, 10, 1, 2, 1, 6, 3, 0, 1, -1, -3, -5, -5, -5, -12, -13, -12],
        'Pts': [22, 18, 17, 17, 16, 16, 15, 15, 14, 13, 13, 12, 12, 11, 11, 10, 8, 5, 4, 2],
        'Pts/MP': [2.44, 2.00, 1.89, 1.89, 1.78, 1.78, 1.67, 1.67, 1.56, 1.44,
                  1.44, 1.33, 1.33, 1.22, 1.22, 1.11, 0.89, 0.56, 0.44, 0.22],
        'xG': [14.6, 11.0, 10.3, 8.6, 15.6, 16.2, 15.8, 7.1, 13.4, 17.6,
              13.4, 11.9, 13.2, 12.3, 11.5, 6.8, 9.0, 9.1, 7.6, 9.2],
        'xGA': [5.2, 11.1, 10.5, 10.4, 9.0, 13.6, 12.0, 11.8, 13.1, 11.7,
               12.2, 7.0, 11.8, 14.0, 9.0, 18.1, 12.8, 14.7, 14.8, 11.4],
        'xGD': [9.3, -0.1, -0.2, -1.8, 6.6, 2.6, 3.8, -4.7, 0.3, 5.9,
               1.2, 4.9, 1.4, -1.7, 2.5, -11.3, -3.8, -5.5, -7.2, -2.2],
        'Attendance': [60144, 11164, 61039, 46335, 52311, 73965, 60362, 41242,
                      39640, 25068, 17014, 52183, 31518, 51883, 36722, 21521,
                      27172, 30221, 62457, 30231],
        'Top_Scorer': ['Viktor Gyökeres - 3', 'Antoine Semenyo - 6', 'Richarlison, Nicky van de Ven - 3',
                      'Wilson Isidor - 4', 'Erling Haaland - 11', 'Bryan Mbeumo - 4',
                      'Mohamed Salah, Cody Gakpo - 3', 'Emi Buendía, Donyell Malen - 2',
                      'Enzo Fernández, Moisés Caicedo - 3', 'Jean-Philippe Mateta - 5',
                      'Thiago - 6', 'Nick Woltemade - 4', 'Danny Welbeck - 5',
                      'Iliman Ndiaye - 3', 'Joe Rodon, Noah Okafor - 2', 'Jaidon Anthony - 4',
                      'Raúl Jiménez, Alex Iwobi - 1', 'Chris Wood - 2', 'Jarrod Bowen - 3',
                      'Hwang Hee-chan, Santiago Bueno - 1']
    }

    df = pd.DataFrame(data)

    # คำนวณค่าเพิ่มเติม
    df['Win_Rate'] = (df['W'] / df['MP'] * 100).round(1)
    df['xG_diff'] = (df['GF'] - df['xG']).round(1)
    df['xGA_diff'] = (df['GA'] - df['xGA']).round(1)
    df['Goals_per_Match'] = (df['GF'] / df['MP']).round(2)
    df['Conceded_per_Match'] = (df['GA'] / df['MP']).round(2)

    return df

def analyze_insights(df):
    """
    วิเคราะห์หา insights ที่น่าสนใจจากข้อมูล
    """
    insights = []

    # 1. ทีมที่นำตาราง
    top_team = df.iloc[0]
    insights.append(f"🏆 ทีมอันดับ 1: {top_team['Squad']} ({top_team['Pts']} คะแนน, {top_team['W']} ชนะ)")

    # 2. ทีมที่ทำประตูได้มากที่สุด
    top_scorer_team = df.loc[df['GF'].idxmax()]
    insights.append(f"⚽ ทำประตูมากสุด: {top_scorer_team['Squad']} ({top_scorer_team['GF']} ประตู)")

    # 3. ทีมที่รับประตูน้อยที่สุด
    best_defense = df.loc[df['GA'].idxmin()]
    insights.append(f"🛡️ ป้องกันดีสุด: {best_defense['Squad']} (เสีย {best_defense['GA']} ประตูเท่านั้น)")

    # 4. ทีมที่มี xG diff สูงสุด (โชคดี/ทำได้เกินคาด)
    lucky_team = df.loc[df['xG_diff'].idxmax()]
    if lucky_team['xG_diff'] > 2:
        insights.append(f"⚡ ทำประตูเกินคาด: {lucky_team['Squad']} (+{lucky_team['xG_diff']:.1f} เทียบกับ xG)")

    # 5. ทีมที่มี xGA diff ติดลบมาก (โชคดีในการป้องกัน)
    lucky_defense = df.loc[df['xGA_diff'].idxmin()]
    if lucky_defense['xGA_diff'] < -2:
        insights.append(f"🍀 เสียประตูน้อยกว่าที่คาด: {lucky_defense['Squad']} ({lucky_defense['xGA_diff']:.1f} เทียบกับ xGA)")

    # 6. ทีมที่มี Goal Difference ดีที่สุด
    best_gd = df.loc[df['GD'].idxmax()]
    insights.append(f"📊 ผลต่างประตูดีสุด: {best_gd['Squad']} (+{best_gd['GD']})")

    # 7. ทีมที่มีผู้ชมเยอะที่สุด
    most_attendance = df.loc[df['Attendance'].idxmax()]
    insights.append(f"👥 ผู้ชมเฉลี่ยสูงสุด: {most_attendance['Squad']} ({most_attendance['Attendance']:,} คน)")

    # 8. ทีมที่มีอัตราชนะสูงสุด
    best_win_rate = df.loc[df['Win_Rate'].idxmax()]
    insights.append(f"📈 อัตราชนะสูงสุด: {best_win_rate['Squad']} ({best_win_rate['Win_Rate']:.1f}%)")

    # 9. ทีมที่ดิ้นรนหนัก (xGD ดีแต่คะแนนน้อย)
    df['xPts_estimate'] = (df['W'] * 3 + df['D'])  # คะแนนจริง
    unlucky_teams = df[df['xGD'] > 0].nsmallest(3, 'Pts')
    if not unlucky_teams.empty:
        insights.append(f"😓 ทีมที่น่าจะได้คะแนนมากกว่านี้: {', '.join(unlucky_teams['Squad'].tolist())}")

    return insights

def create_visualizations(df):
    """
    สร้างกราฟวิเคราะห์ที่หลากหลาย
    """

    # 1. ตารางคะแนน Top 10
    fig1 = go.Figure()

    top10 = df.head(10).sort_values('Pts', ascending=True)

    fig1.add_trace(go.Bar(
        y=top10['Squad'],
        x=top10['Pts'],
        orientation='h',
        marker=dict(
            color=top10['Pts'],
            colorscale='Viridis',
            showscale=True
        ),
        text=top10['Pts'],
        textposition='outside',
        hovertemplate='<b>%{y}</b><br>คะแนน: %{x}<br><extra></extra>'
    ))

    fig1.update_layout(
        title='🏆 Premier League - Top 10 Teams',
        xaxis_title='คะแนน (Points)',
        yaxis_title='ทีม',
        height=600,
        template='plotly_white'
    )

    # 2. Goals Scored vs Conceded (Bubble Chart)
    fig2 = px.scatter(df,
                      x='GF',
                      y='GA',
                      size='Pts',
                      color='Pts',
                      hover_name='Squad',
                      size_max=50,
                      color_continuous_scale='RdYlGn',
                      title='⚽ Goals Scored vs Goals Conceded (ขนาดวง = คะแนน)')

    fig2.add_shape(type='line', x0=0, y0=0, x1=25, y1=25,
                   line=dict(color='gray', dash='dash'),
                   name='เสมอ')

    fig2.update_layout(
        xaxis_title='ประตูที่ทำได้ (GF)',
        yaxis_title='ประตูที่เสีย (GA)',
        height=600
    )

    # 3. Actual Goals vs Expected Goals (xG)
    fig3 = go.Figure()

    # เรียงตามคะแนน
    df_sorted = df.sort_values('Pts', ascending=True)

    fig3.add_trace(go.Bar(
        name='Actual Goals (GF)',
        x=df_sorted['Squad'],
        y=df_sorted['GF'],
        marker_color='steelblue'
    ))

    fig3.add_trace(go.Scatter(
        name='Expected Goals (xG)',
        x=df_sorted['Squad'],
        y=df_sorted['xG'],
        mode='markers',
        marker=dict(size=10, color='red', symbol='diamond')
    ))

    fig3.update_layout(
        title='📈 Actual Goals vs Expected Goals (xG)',
        xaxis_title='ทีม',
        yaxis_title='จำนวนประตู',
        xaxis_tickangle=-45,
        height=600,
        template='plotly_white',
        barmode='group'
    )

    # 4. Win/Draw/Loss Distribution
    fig4 = go.Figure()

    df_sorted = df.sort_values('Pts', ascending=False)

    fig4.add_trace(go.Bar(
        name='ชนะ (W)',
        x=df_sorted['Squad'],
        y=df_sorted['W'],
        marker_color='green'
    ))

    fig4.add_trace(go.Bar(
        name='เสมอ (D)',
        x=df_sorted['Squad'],
        y=df_sorted['D'],
        marker_color='orange'
    ))

    fig4.add_trace(go.Bar(
        name='แพ้ (L)',
        x=df_sorted['Squad'],
        y=df_sorted['L'],
        marker_color='red'
    ))

    fig4.update_layout(
        title='📊 การกระจายผลการแข่งขัน (W/D/L)',
        xaxis_title='ทีม',
        yaxis_title='จำนวนเกม',
        barmode='stack',
        xaxis_tickangle=-45,
        height=600
    )

    # 5. xG Difference (Over/Under Performance)
    fig5 = go.Figure()

    df_sorted = df.sort_values('xG_diff', ascending=True)

    colors = ['red' if x < 0 else 'green' for x in df_sorted['xG_diff']]

    fig5.add_trace(go.Bar(
        x=df_sorted['Squad'],
        y=df_sorted['xG_diff'],
        marker_color=colors,
        text=df_sorted['xG_diff'].round(1),
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>xG Diff: %{y:.1f}<extra></extra>'
    ))

    fig5.add_hline(y=0, line_dash="dash", line_color="black")

    fig5.update_layout(
        title='🎯 Actual Goals - Expected Goals (มากกว่า xG = สีเขียว, น้อยกว่า = สีแดง)',
        xaxis_title='ทีม',
        yaxis_title='ผลต่าง (GF - xG)',
        xaxis_tickangle=-45,
        height=600
    )

    # 6. Attendance vs Performance
    fig6 = px.scatter(df,
                     x='Attendance',
                     y='Pts',
                     size='GF',
                     color='GD',
                     hover_name='Squad',
                     title='👥 ผู้ชม vs ผลงาน (ขนาด = จำนวนประตูที่ทำได้)',
                     color_continuous_scale='RdYlGn')

    fig6.update_layout(
        xaxis_title='จำนวนผู้ชมเฉลี่ย',
        yaxis_title='คะแนน',
        height=600
    )

    # 7. Goal Difference Ranking
    fig7 = px.bar(df.sort_values('GD', ascending=True),
                  x='GD',
                  y='Squad',
                  orientation='h',
                  color='GD',
                  color_continuous_scale='RdYlGn',
                  title='🔢 Goal Difference (ผลต่างประตู)')

    fig7.update_layout(height=700)

    return fig1, fig2, fig3, fig4, fig5, fig6, fig7

# Main execution
if __name__ == "__main__":
    print("="*70)
    print("⚽ PREMIER LEAGUE DATA ANALYSIS")
    print("="*70)

    url = "https://fbref.com/en/comps/9/Premier-League-Stats"

    print("\n🔄 กำลังดึงข้อมูล Premier League...")
    df = scrape_premier_league_data(url)

    print("\n" + "="*70)
    print("🔍 KEY INSIGHTS - ข้อมูลเชิงลึกที่น่าสนใจ")
    print("="*70)

    insights = analyze_insights(df)
    for i, insight in enumerate(insights, 1):
        print(f"{i}. {insight}")

    print("\n" + "="*70)
    print("📊 สถิติเพิ่มเติม")
    print("="*70)
    print(f"• ค่าเฉลี่ยประตูต่อเกมทั้งลีก: {df['Goals_per_Match'].mean():.2f}")
    print(f"• ทีมที่ทำประตูมากที่สุด 3 อันดับ: {', '.join(df.nlargest(3, 'GF')['Squad'].tolist())}")
    print(f"• ทีมที่เสียประตูน้อยที่สุด 3 อันดับ: {', '.join(df.nsmallest(3, 'GA')['Squad'].tolist())}")
    print(f"• ทีมที่มีผู้ชมมากที่สุด: {df.loc[df['Attendance'].idxmax(), 'Squad']} ({df['Attendance'].max():,} คน)")

    print("\n" + "="*70)
    print("📈 กำลังสร้างกราฟ...")
    print("="*70)

    fig1, fig2, fig3, fig4, fig5, fig6, fig7 = create_visualizations(df)

    # แสดงกราฟทั้งหมด
    fig1.show()
    fig2.show()
    fig3.show()
    fig4.show()
    fig5.show()
    fig6.show()
    fig7.show()

    # แสดงตารางข้อมูล
    print("\n" + "="*70)
    print("📋 ตารางข้อมูลทั้งหมด")
    print("="*70)
    print(df[['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'xG', 'xGA']].to_string(index=False))

    # บันทึกข้อมูล
    df.to_csv('premier_league_stats_2024_25.csv', index=False, encoding='utf-8-sig')
    print("\n✅ บันทึกข้อมูลเป็นไฟล์ 'premier_league_stats_2024_25.csv' เรียบร้อยแล้ว!")

    # สรุปทีมที่น่าสนใจ
    print("\n" + "="*70)
    print("🌟 ทีมที่น่าจับตามอง")
    print("="*70)

    # ทีมที่ทำได้ดีกว่าที่คาด
    overperformers = df[df['xG_diff'] > 2].sort_values('xG_diff', ascending=False)
    if not overperformers.empty:
        print("\n🚀 ทีมที่ทำได้เหนือความคาด (xG):")
        for _, team in overperformers.iterrows():
            print(f"   • {team['Squad']}: ทำได้ {team['GF']} ประตู (คาดหวัง {team['xG']:.1f})")

    # ทีมที่น่าเป็นห่วง
    underperformers = df[df['xG_diff'] < -2].sort_values('xG_diff')
    if not underperformers.empty:
        print("\n⚠️ ทีมที่ยังทำได้ไม่ดีเท่าที่ควร:")
        for _, team in underperformers.iterrows():
            print(f"   • {team['Squad']}: ทำได้ {team['GF']} ประตู (คาดหวัง {team['xG']:.1f})")

⚽ PREMIER LEAGUE DATA ANALYSIS

🔄 กำลังดึงข้อมูล Premier League...
<selenium.webdriver.chrome.webdriver.WebDriver (session="9af9c18e9dcc8d8f6bd28270ea14920f")>


/tmp/ipython-input-893733226.py:42: FutureWarning:

Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



✅ ดึงข้อมูลสำเร็จ! พบ 1 ทีม
❌ Error: 'list' object has no attribute 'columns'
💡 ใช้ข้อมูลจากภาพที่แนบมา

🔍 KEY INSIGHTS - ข้อมูลเชิงลึกที่น่าสนใจ
1. 🏆 ทีมอันดับ 1: Arsenal (22 คะแนน, 7 ชนะ)
2. ⚽ ทำประตูมากสุด: Tottenham (17 ประตู)
3. 🛡️ ป้องกันดีสุด: Arsenal (เสีย 3 ประตูเท่านั้น)
4. ⚡ ทำประตูเกินคาด: Tottenham (+6.7 เทียบกับ xG)
5. 🍀 เสียประตูน้อยกว่าที่คาด: Aston Villa (-3.8 เทียบกับ xGA)
6. 📊 ผลต่างประตูดีสุด: Arsenal (+13)
7. 👥 ผู้ชมเฉลี่ยสูงสุด: Manchester Utd (73,965 คน)
8. 📈 อัตราชนะสูงสุด: Arsenal (77.8%)
9. 😓 ทีมที่น่าจะได้คะแนนมากกว่านี้: Leeds United, Newcastle Utd, Brighton

📊 สถิติเพิ่มเติม
• ค่าเฉลี่ยประตูต่อเกมทั้งลีก: 1.34
• ทีมที่ทำประตูมากที่สุด 3 อันดับ: Tottenham, Manchester City, Chelsea
• ทีมที่เสียประตูน้อยที่สุด 3 อันดับ: Arsenal, Tottenham, Sunderland
• ทีมที่มีผู้ชมมากที่สุด: Manchester Utd (73,965 คน)

📈 กำลังสร้างกราฟ...



📋 ตารางข้อมูลทั้งหมด
 Rk             Squad  MP  W  D  L  GF  GA  GD  Pts   xG  xGA
  1           Arsenal   9  7  1  1  16   3  13   22 14.6  5.2
  2       Bournemouth   9  5  3  1  16  11   5   18 11.0 11.1
  3         Tottenham   9  5  2  2  17   7  10   17 10.3 10.5
  4        Sunderland   9  5  2  2  11   7   4   17  8.6 10.4
  5   Manchester City   9  5  1  3  17   7  10   16 15.6  9.0
  6    Manchester Utd   9  5  1  3  15  14   1   16 16.2 13.6
  7         Liverpool   9  5  0  4  16  14   2   15 15.8 12.0
  8       Aston Villa   9  4  3  2   9   8   1   15  7.1 11.8
  9           Chelsea   9  4  2  3  17  11   6   14 13.4 13.1
 10    Crystal Palace   9  3  4  2  12   9   3   13 17.6 11.7
 11         Brentford   9  4  1  4  14  14   0   13 13.4 12.2
 12     Newcastle Utd   9  3  3  3   9   8   1   12 11.9  7.0
 13          Brighton   9  3  3  3  14  15  -1   12 13.2 11.8
 14           Everton   9  3  2  4   9  12  -3   11 12.3 14.0
 15      Leeds United   9  3  2  4   9  14  -5  

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ฟังก์ชันสำหรับดึงข้อมูลจาก FBref
def scrape_premier_league_data(url):
    """
    Scrape Premier League statistics from FBref
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 2. Fetch the HTML content
    try:
      response = requests.get(url, headers=headers)
      response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
      html_content = response.text
    except requests.exceptions.RequestException as e:
      print(f"Error fetching the URL: {e}")
    # Continue execution even if there's an error, although parsing will fail
    html_content = None

    # Correct table by inspect
    # หาตารางสถิติทีม (League Table)
    table = soup.find('table', {'id': 'results2025-202691_overall'})

    if table:
        # อ่านตารางด้วย pandas
        df = pd.read_html(str(table))[0]

        # ทำความสะอาดข้อมูล
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = ['_'.join(col).strip() if col[1] else col[0]
                         for col in df.columns.values]

        return df
    else:
        print("ไม่พบตาราง ลองใช้ข้อมูล demo")
        return create_demo_data()


# Just a DEMO ! : This is not necessary.
def create_demo_data():
    """
    สร้างข้อมูล demo สำหรับการวิเคราะห์
    """
    data = {
        'Squad': ['Liverpool', 'Arsenal', 'Man City', 'Chelsea', 'Aston Villa',
                 'Brighton', 'Newcastle', 'Tottenham', 'Man United', 'West Ham',
                 'Fulham', 'Brentford', 'Wolves', 'Nottm Forest', 'Bournemouth',
                 'Everton', 'Leicester', 'Crystal Palace', 'Ipswich', 'Southampton'],
        'MP': [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        'W': [8, 7, 7, 6, 6, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 3, 2, 2, 1, 1],
        'D': [2, 2, 2, 3, 3, 3, 3, 1, 3, 2, 2, 2, 4, 4, 3, 2, 3, 2, 3, 1],
        'L': [0, 1, 1, 1, 1, 2, 2, 4, 3, 4, 4, 4, 3, 3, 4, 5, 5, 6, 6, 8],
        'GF': [19, 18, 20, 20, 17, 16, 11, 18, 11, 13, 13, 18, 14, 11, 12, 10, 12, 8, 9, 7],
        'GA': [4, 8, 9, 11, 11, 12, 10, 10, 10, 16, 12, 18, 18, 13, 16, 15, 18, 12, 18, 18],
        'GD': [15, 10, 11, 9, 6, 4, 1, 8, 1, -3, 1, 0, -4, -2, -4, -5, -6, -4, -9, -11],
        'Pts': [26, 23, 23, 21, 21, 18, 18, 16, 15, 14, 14, 14, 13, 13, 12, 11, 9, 8, 6, 4],
        'xG': [20.5, 19.2, 22.1, 18.5, 16.8, 15.3, 12.4, 19.8, 12.2, 14.1, 13.8, 17.2, 13.5, 10.8, 11.9, 9.7, 11.5, 9.2, 8.8, 8.1],
        'xGA': [7.2, 9.5, 10.2, 12.3, 11.8, 13.1, 11.5, 12.8, 11.9, 15.8, 13.2, 16.5, 16.8, 12.9, 14.7, 14.2, 17.3, 13.5, 17.2, 19.5]
    }
    return pd.DataFrame(data)

def analyze_data(df):
    """
    วิเคราะห์ข้อมูลและสร้าง insights
    """
    insights = []

    # 1. ทีมที่มีประสิทธิภาพการทำประตูดีที่สุด
    df['Goals_per_Match'] = df['GF'] / df['MP']
    df['Conceded_per_Match'] = df['GA'] / df['MP']

    top_scorer = df.nlargest(1, 'GF')['Squad'].values[0]
    insights.append(f"🎯 ทีมที่ทำประตูได้มากที่สุด: {top_scorer} ({df[df['Squad']==top_scorer]['GF'].values[0]} ประตู)")

    # 2. ทีมที่มีการป้องกันดีที่สุด
    best_defense = df.nsmallest(1, 'GA')['Squad'].values[0]
    insights.append(f"🛡️ ทีมที่เสียประตูน้อยที่สุด: {best_defense} ({df[df['Squad']==best_defense]['GA'].values[0]} ประตู)")

    # 3. ทีมที่มี Goal Difference ดีที่สุด
    best_gd = df.nlargest(1, 'GD')['Squad'].values[0]
    insights.append(f"📊 ผลต่างประตูดีที่สุด: {best_gd} (+{df[df['Squad']==best_gd]['GD'].values[0]})")

    # 4. วิเคราะห์ xG (Expected Goals)
    if 'xG' in df.columns and 'xGA' in df.columns:
        df['xG_diff'] = df['GF'] - df['xG']
        df['xGA_diff'] = df['GA'] - df['xGA']

        overperformer = df.nlargest(1, 'xG_diff')['Squad'].values[0]
        insights.append(f"⚡ ทีมที่ทำประตูได้มากกว่าที่คาด (xG): {overperformer}")

        lucky_defense = df.nsmallest(1, 'xGA_diff')['Squad'].values[0]
        insights.append(f"🍀 ทีมที่เสียประตูน้อยกว่าที่คาด (xGA): {lucky_defense}")

    # 5. อัตราชนะ
    df['Win_Rate'] = (df['W'] / df['MP']) * 100

    # 6. คะแนนต่อเกม
    df['Points_per_Game'] = df['Pts'] / df['MP']

    return df, insights

def create_visualizations(df):
    """
    สร้างกราฟวิเคราะห์ต่างๆ
    """

    # 1. Top 10 Teams by Points
    fig1 = px.bar(df.nlargest(10, 'Pts').sort_values('Pts', ascending=True),
                  x='Pts', y='Squad', orientation='h',
                  title='🏆 Top 10 Teams - Total Points',
                  labels={'Pts': 'Points', 'Squad': 'Team'},
                  color='Pts',
                  color_continuous_scale='Viridis')
    fig1.update_layout(height=500, showlegend=False)

    # 2. Goals Scored vs Conceded
    fig2 = px.scatter(df, x='GF', y='GA', text='Squad',
                      size='Pts', color='Pts',
                      title='⚽ Goals Scored vs Goals Conceded',
                      labels={'GF': 'Goals For', 'GA': 'Goals Against'},
                      color_continuous_scale='RdYlGn_r')
    fig2.update_traces(textposition='top center')
    fig2.update_layout(height=600)

    # 3. Expected Goals Analysis (ถ้ามีข้อมูล xG)
    if 'xG' in df.columns and 'xGA' in df.columns:
        fig3 = go.Figure()

        fig3.add_trace(go.Scatter(
            x=df['Squad'], y=df['xG'],
            name='Expected Goals (xG)',
            mode='markers',
            marker=dict(size=10, color='lightblue')
        ))

        fig3.add_trace(go.Scatter(
            x=df['Squad'], y=df['GF'],
            name='Actual Goals',
            mode='markers',
            marker=dict(size=10, color='darkblue')
        ))

        fig3.update_layout(
            title='📈 Actual Goals vs Expected Goals (xG)',
            xaxis_title='Team',
            yaxis_title='Goals',
            height=600,
            xaxis_tickangle=-45
        )
    else:
        fig3 = None

    # 4. Win/Draw/Loss Distribution
    fig4 = go.Figure()

    fig4.add_trace(go.Bar(name='Wins', x=df['Squad'], y=df['W'], marker_color='green'))
    fig4.add_trace(go.Bar(name='Draws', x=df['Squad'], y=df['D'], marker_color='orange'))
    fig4.add_trace(go.Bar(name='Losses', x=df['Squad'], y=df['L'], marker_color='red'))

    fig4.update_layout(
        title='📊 Match Results Distribution',
        xaxis_title='Team',
        yaxis_title='Number of Matches',
        barmode='stack',
        height=600,
        xaxis_tickangle=-45
    )

    # 5. Points per Game
    fig5 = px.bar(df.sort_values('Points_per_Game', ascending=False),
                  x='Squad', y='Points_per_Game',
                  title='💯 Points per Game Average',
                  labels={'Points_per_Game': 'Points/Game', 'Squad': 'Team'},
                  color='Points_per_Game',
                  color_continuous_scale='Blues')
    fig5.update_layout(height=600, xaxis_tickangle=-45)

    return fig1, fig2, fig3, fig4, fig5

# Main execution
if __name__ == "__main__":
    url = "https://fbref.com/en/comps/9/Premier-League-Stats"

    print("🔄 กำลังดึงข้อมูล Premier League...")
    df = scrape_premier_league_data(url)

    print("\n📊 กำลังวิเคราะห์ข้อมูล...")
    df, insights = analyze_data(df)

    print("\n" + "="*60)
    print("🔍 INSIGHTS - ข้อมูลที่น่าสนใจ")
    print("="*60)
    for insight in insights:
        print(insight)

    print("\n" + "="*60)
    print("📈 สร้างกราฟวิเคราะห์...")
    print("="*60)

    fig1, fig2, fig3, fig4, fig5 = create_visualizations(df)

    # แสดงกราฟ
    fig1.show()
    fig2.show()
    if fig3:
        fig3.show()
    fig4.show()
    fig5.show()

    # แสดง DataFrame
    print("\n📋 ตารางสถิติทั้งหมด:")
    print(df.to_string())

    df.head(5)

    # บันทึกข้อมูลเป็น CSV (optional)
    df.to_csv('premier_league_stats.csv', index=False, encoding='utf-8-sig')
    print("\n✅ บันทึกข้อมูลเป็นไฟล์ premier_league_stats.csv เรียบร้อยแล้ว!")

🔄 กำลังดึงข้อมูล Premier League...
Error fetching the URL: 403 Client Error: Forbidden for url: https://fbref.com/en/comps/9/Premier-League-Stats
ไม่พบตาราง ลองใช้ข้อมูล demo

📊 กำลังวิเคราะห์ข้อมูล...

🔍 INSIGHTS - ข้อมูลที่น่าสนใจ
🎯 ทีมที่ทำประตูได้มากที่สุด: Man City (20 ประตู)
🛡️ ทีมที่เสียประตูน้อยที่สุด: Liverpool (4 ประตู)
📊 ผลต่างประตูดีที่สุด: Liverpool (+15)
⚡ ทีมที่ทำประตูได้มากกว่าที่คาด (xG): Chelsea
🍀 ทีมที่เสียประตูน้อยกว่าที่คาด (xGA): Liverpool

📈 สร้างกราฟวิเคราะห์...



📋 ตารางสถิติทั้งหมด:
             Squad  MP  W  D  L  GF  GA  GD  Pts    xG   xGA  Goals_per_Match  Conceded_per_Match  xG_diff  xGA_diff  Win_Rate  Points_per_Game
0        Liverpool  10  8  2  0  19   4  15   26  20.5   7.2              1.9                 0.4     -1.5      -3.2      80.0              2.6
1          Arsenal  10  7  2  1  18   8  10   23  19.2   9.5              1.8                 0.8     -1.2      -1.5      70.0              2.3
2         Man City  10  7  2  1  20   9  11   23  22.1  10.2              2.0                 0.9     -2.1      -1.2      70.0              2.3
3          Chelsea  10  6  3  1  20  11   9   21  18.5  12.3              2.0                 1.1      1.5      -1.3      60.0              2.1
4      Aston Villa  10  6  3  1  17  11   6   21  16.8  11.8              1.7                 1.1      0.2      -0.8      60.0              2.1
5         Brighton  10  5  3  2  16  12   4   18  15.3  13.1              1.6                 1.2      0.7      -1

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ฟังก์ชันสำหรับดึงข้อมูลจาก FBref
def scrape_premier_league_data(url):
    """
    Scrape Premier League statistics from FBref
    """
    import time

    # Headers ที่ครบถ้วนเพื่อหลีกเลี่ยง 403 Error
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.9,th;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Cache-Control': 'max-age=0',
        'Referer': 'https://www.google.com/'
    }

    try:
        # เพิ่ม timeout และ delay เล็กน้อย
        time.sleep(2)  # รอ 2 วินาทีก่อน request

        session = requests.Session()
        session.headers.update(headers)

        response = session.get(url, timeout=30)
        response.raise_for_status()  # จะ raise exception ถ้าเจอ error

        soup = BeautifulSoup(response.content, 'html.parser')

        # ลองหาตารางหลายรูปแบบ
        table = None
        possible_ids = [
            'results2025-202691_overall',
            'results2024-202591_overall',
            'stats_squads_standard_for'
        ]

        for table_id in possible_ids:
            table = soup.find('table', {'id': table_id})
            if table:
                print(f"✅ พบตาราง: {table_id}")
                break

        # ถ้าไม่เจอ ลองหา table ทั่วไป
        if not table:
            tables = soup.find_all('table')
            if tables:
                table = tables[0]  # ใช้ตารางแรก
                print(f"✅ ใช้ตารางแรกที่พบ")

        if table:
            # อ่านตารางด้วย pandas
            df = pd.read_html(str(table))[0]

            # ทำความสะอาดข้อมูล
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = ['_'.join(col).strip() if col[1] else col[0]
                             for col in df.columns.values]

            print(f"✅ ดึงข้อมูลสำเร็จ! พบ {len(df)} ทีม")
            return df
        else:
            print("❌ ไม่พบตาราง ใช้ข้อมูล demo แทน")
            return create_demo_data()

    except requests.exceptions.HTTPError as e:
        print(f"❌ HTTP Error: {e}")
        print("💡 เว็บไซต์บล็อกการเข้าถึง ใช้ข้อมูล demo แทน")
        return create_demo_data()

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        print("💡 ใช้ข้อมูล demo แทน")
        return create_demo_data()

def create_demo_data():
    """
    สร้างข้อมูล demo สำหรับการวิเคราะห์
    """
    data = {
        'Squad': ['Liverpool', 'Arsenal', 'Man City', 'Chelsea', 'Aston Villa',
                 'Brighton', 'Newcastle', 'Tottenham', 'Man United', 'West Ham',
                 'Fulham', 'Brentford', 'Wolves', 'Nottm Forest', 'Bournemouth',
                 'Everton', 'Leicester', 'Crystal Palace', 'Ipswich', 'Southampton'],
        'MP': [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        'W': [8, 7, 7, 6, 6, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 3, 2, 2, 1, 1],
        'D': [2, 2, 2, 3, 3, 3, 3, 1, 3, 2, 2, 2, 4, 4, 3, 2, 3, 2, 3, 1],
        'L': [0, 1, 1, 1, 1, 2, 2, 4, 3, 4, 4, 4, 3, 3, 4, 5, 5, 6, 6, 8],
        'GF': [19, 18, 20, 20, 17, 16, 11, 18, 11, 13, 13, 18, 14, 11, 12, 10, 12, 8, 9, 7],
        'GA': [4, 8, 9, 11, 11, 12, 10, 10, 10, 16, 12, 18, 18, 13, 16, 15, 18, 12, 18, 18],
        'GD': [15, 10, 11, 9, 6, 4, 1, 8, 1, -3, 1, 0, -4, -2, -4, -5, -6, -4, -9, -11],
        'Pts': [26, 23, 23, 21, 21, 18, 18, 16, 15, 14, 14, 14, 13, 13, 12, 11, 9, 8, 6, 4],
        'xG': [20.5, 19.2, 22.1, 18.5, 16.8, 15.3, 12.4, 19.8, 12.2, 14.1, 13.8, 17.2, 13.5, 10.8, 11.9, 9.7, 11.5, 9.2, 8.8, 8.1],
        'xGA': [7.2, 9.5, 10.2, 12.3, 11.8, 13.1, 11.5, 12.8, 11.9, 15.8, 13.2, 16.5, 16.8, 12.9, 14.7, 14.2, 17.3, 13.5, 17.2, 19.5]
    }
    return pd.DataFrame(data)

def analyze_data(df):
    """
    วิเคราะห์ข้อมูลและสร้าง insights
    """
    insights = []

    # 1. ทีมที่มีประสิทธิภาพการทำประตูดีที่สุด
    df['Goals_per_Match'] = df['GF'] / df['MP']
    df['Conceded_per_Match'] = df['GA'] / df['MP']

    top_scorer = df.nlargest(1, 'GF')['Squad'].values[0]
    insights.append(f"🎯 ทีมที่ทำประตูได้มากที่สุด: {top_scorer} ({df[df['Squad']==top_scorer]['GF'].values[0]} ประตู)")

    # 2. ทีมที่มีการป้องกันดีที่สุด
    best_defense = df.nsmallest(1, 'GA')['Squad'].values[0]
    insights.append(f"🛡️ ทีมที่เสียประตูน้อยที่สุด: {best_defense} ({df[df['Squad']==best_defense]['GA'].values[0]} ประตู)")

    # 3. ทีมที่มี Goal Difference ดีที่สุด
    best_gd = df.nlargest(1, 'GD')['Squad'].values[0]
    insights.append(f"📊 ผลต่างประตูดีที่สุด: {best_gd} (+{df[df['Squad']==best_gd]['GD'].values[0]})")

    # 4. วิเคราะห์ xG (Expected Goals)
    if 'xG' in df.columns and 'xGA' in df.columns:
        df['xG_diff'] = df['GF'] - df['xG']
        df['xGA_diff'] = df['GA'] - df['xGA']

        overperformer = df.nlargest(1, 'xG_diff')['Squad'].values[0]
        insights.append(f"⚡ ทีมที่ทำประตูได้มากกว่าที่คาด (xG): {overperformer}")

        lucky_defense = df.nsmallest(1, 'xGA_diff')['Squad'].values[0]
        insights.append(f"🍀 ทีมที่เสียประตูน้อยกว่าที่คาด (xGA): {lucky_defense}")

    # 5. อัตราชนะ
    df['Win_Rate'] = (df['W'] / df['MP']) * 100

    # 6. คะแนนต่อเกม
    df['Points_per_Game'] = df['Pts'] / df['MP']

    return df, insights

def create_visualizations(df):
    """
    สร้างกราฟวิเคราะห์ต่างๆ
    """

    # 1. Top 10 Teams by Points
    fig1 = px.bar(df.nlargest(10, 'Pts').sort_values('Pts', ascending=True),
                  x='Pts', y='Squad', orientation='h',
                  title='🏆 Top 10 Teams - Total Points',
                  labels={'Pts': 'Points', 'Squad': 'Team'},
                  color='Pts',
                  color_continuous_scale='Viridis')
    fig1.update_layout(height=500, showlegend=False)

    # 2. Goals Scored vs Conceded
    fig2 = px.scatter(df, x='GF', y='GA', text='Squad',
                      size='Pts', color='Pts',
                      title='⚽ Goals Scored vs Goals Conceded',
                      labels={'GF': 'Goals For', 'GA': 'Goals Against'},
                      color_continuous_scale='RdYlGn_r')
    fig2.update_traces(textposition='top center')
    fig2.update_layout(height=600)

    # 3. Expected Goals Analysis (ถ้ามีข้อมูล xG)
    if 'xG' in df.columns and 'xGA' in df.columns:
        fig3 = go.Figure()

        fig3.add_trace(go.Scatter(
            x=df['Squad'], y=df['xG'],
            name='Expected Goals (xG)',
            mode='markers',
            marker=dict(size=10, color='lightblue')
        ))

        fig3.add_trace(go.Scatter(
            x=df['Squad'], y=df['GF'],
            name='Actual Goals',
            mode='markers',
            marker=dict(size=10, color='darkblue')
        ))

        fig3.update_layout(
            title='📈 Actual Goals vs Expected Goals (xG)',
            xaxis_title='Team',
            yaxis_title='Goals',
            height=600,
            xaxis_tickangle=-45
        )
    else:
        fig3 = None

    # 4. Win/Draw/Loss Distribution
    fig4 = go.Figure()

    fig4.add_trace(go.Bar(name='Wins', x=df['Squad'], y=df['W'], marker_color='green'))
    fig4.add_trace(go.Bar(name='Draws', x=df['Squad'], y=df['D'], marker_color='orange'))
    fig4.add_trace(go.Bar(name='Losses', x=df['Squad'], y=df['L'], marker_color='red'))

    fig4.update_layout(
        title='📊 Match Results Distribution',
        xaxis_title='Team',
        yaxis_title='Number of Matches',
        barmode='stack',
        height=600,
        xaxis_tickangle=-45
    )

    # 5. Points per Game
    fig5 = px.bar(df.sort_values('Points_per_Game', ascending=False),
                  x='Squad', y='Points_per_Game',
                  title='💯 Points per Game Average',
                  labels={'Points_per_Game': 'Points/Game', 'Squad': 'Team'},
                  color='Points_per_Game',
                  color_continuous_scale='Blues')
    fig5.update_layout(height=600, xaxis_tickangle=-45)

    return fig1, fig2, fig3, fig4, fig5

# Main execution
if __name__ == "__main__":
    url = "https://fbref.com/en/comps/9/Premier-League-Stats"

    print("🔄 กำลังดึงข้อมูล Premier League...")
    df = scrape_premier_league_data(url)

    print("\n📊 กำลังวิเคราะห์ข้อมูล...")
    df, insights = analyze_data(df)

    print("\n" + "="*60)
    print("🔍 INSIGHTS - ข้อมูลที่น่าสนใจ")
    print("="*60)
    for insight in insights:
        print(insight)

    print("\n" + "="*60)
    print("📈 สร้างกราฟวิเคราะห์...")
    print("="*60)

    fig1, fig2, fig3, fig4, fig5 = create_visualizations(df)

    # แสดงกราฟ
    fig1.show()
    fig2.show()
    if fig3:
        fig3.show()
    fig4.show()
    fig5.show()

    # แสดง DataFrame
    print("\n📋 ตารางสถิติทั้งหมด:")
    print(df.to_string())

    # บันทึกข้อมูลเป็น CSV (optional)
    df.to_csv('premier_league_stats.csv', index=False, encoding='utf-8-sig')
    print("\n✅ บันทึกข้อมูลเป็นไฟล์ premier_league_stats.csv เรียบร้อยแล้ว!")

🔄 กำลังดึงข้อมูล Premier League...
❌ HTTP Error: 403 Client Error: Forbidden for url: https://fbref.com/en/comps/9/Premier-League-Stats
💡 เว็บไซต์บล็อกการเข้าถึง ใช้ข้อมูล demo แทน

📊 กำลังวิเคราะห์ข้อมูล...

🔍 INSIGHTS - ข้อมูลที่น่าสนใจ
🎯 ทีมที่ทำประตูได้มากที่สุด: Man City (20 ประตู)
🛡️ ทีมที่เสียประตูน้อยที่สุด: Liverpool (4 ประตู)
📊 ผลต่างประตูดีที่สุด: Liverpool (+15)
⚡ ทีมที่ทำประตูได้มากกว่าที่คาด (xG): Chelsea
🍀 ทีมที่เสียประตูน้อยกว่าที่คาด (xGA): Liverpool

📈 สร้างกราฟวิเคราะห์...



📋 ตารางสถิติทั้งหมด:
             Squad  MP  W  D  L  GF  GA  GD  Pts    xG   xGA  Goals_per_Match  Conceded_per_Match  xG_diff  xGA_diff  Win_Rate  Points_per_Game
0        Liverpool  10  8  2  0  19   4  15   26  20.5   7.2              1.9                 0.4     -1.5      -3.2      80.0              2.6
1          Arsenal  10  7  2  1  18   8  10   23  19.2   9.5              1.8                 0.8     -1.2      -1.5      70.0              2.3
2         Man City  10  7  2  1  20   9  11   23  22.1  10.2              2.0                 0.9     -2.1      -1.2      70.0              2.3
3          Chelsea  10  6  3  1  20  11   9   21  18.5  12.3              2.0                 1.1      1.5      -1.3      60.0              2.1
4      Aston Villa  10  6  3  1  17  11   6   21  16.8  11.8              1.7                 1.1      0.2      -0.8      60.0              2.1
5         Brighton  10  5  3  2  16  12   4   18  15.3  13.1              1.6                 1.2      0.7      -1

# **Gemini**

In [ ]:
pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 1. Define the URL to scrape
#url = 'http://quotes.toscrape.com/' # A simple, public site for practice

url = 'https://fbref.com/en/comps/9/Premier-League-Stats#all_results2025-202691' # A website designed for scraping

# Add headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


# 2. Fetch the HTML content
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
    html_content = response.text
except requests.exceptions.RequestException as e:
    print(f"Error fetching the URL: {e}")
    # Continue execution even if there's an error, although parsing will fail
    html_content = None


# 3. Parse the HTML content
if html_content: # Only parse if the request was successful
    soup = BeautifulSoup(html_content, 'html.parser')

    # 4. Find the data (Scraping logic)
    # This site has 'quote' div elements, each containing the quote text and author.
    quotes_data = []
    quote_elements = soup.find_all('div', class_='quote')

    for quote_element in quote_elements:
        text = quote_element.find('span', class_='text').text
        author = quote_element.find('small', class_='author').text

        # Store the extracted data
        quotes_data.append({
            'Quote': text,
            'Author': author
        })

    # 5. Load the data into a pandas DataFrame
    df = pd.DataFrame(quotes_data)

    # 6. Print the DataFrame
    print(df)
else:
    print("Could not retrieve HTML content.")

Error fetching the URL: 403 Client Error: Forbidden for url: https://fbref.com/en/comps/9/Premier-League-Stats#all_results2025-202691
Could not retrieve HTML content.


In [ ]:
import pandas as pd
import plotly.express as px

# URL สำหรับหน้าสถิติ Premier League (ตารางคะแนน)
# ใช้ URL หลักที่เสถียรกว่าสำหรับการดึงข้อมูลตารางคะแนน (Standings)
url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

# --- 1. Web Scraping และโหลดข้อมูลใส่ DataFrame ---
try:
    # pd.read_html จะอ่านทุกตารางที่มีในหน้าเว็บ และเราจะเลือกตารางที่มีคอลัมน์ 'Squad' (ตารางคะแนน)
    tables = pd.read_html(url, match='Squad')
    df = tables[0].copy()

except Exception as e:
    print(f"เกิดข้อผิดพลาดในการดึงข้อมูล: {e}")
    # ใช้ตารางจำลองหากการดึงข้อมูลล้มเหลว (สำหรับแสดงผลลัพธ์ในกรณีนี้)
    data = {
        'Team': ['Manchester City', 'Liverpool', 'Arsenal', 'Tottenham', 'Aston Villa'],
        'Points': [75, 73, 72, 60, 58],
        'Goal_Difference': [50, 45, 38, 15, 10]
    }
    df = pd.DataFrame(data)
    print("ใช้ตารางจำลองเพื่อสาธิตการทำงาน")


# --- 2. การทำความสะอาดและเตรียมข้อมูล ---
if not df.empty:

    # 2.1 จัดการชื่อคอลัมน์แบบ Multi-level (ถ้ามี) ซึ่งเป็นเรื่องปกติของ FBRef
    if isinstance(df.columns, pd.MultiIndex):
        new_columns = []
        for col in df.columns.tolist():
            if 'Unnamed' in col[1]:
                new_columns.append(col[0])
            else:
                new_columns.append(col[1])
        df.columns = new_columns

    # 2.2 เปลี่ยนชื่อคอลัมน์หลักให้เข้าใจง่าย
    df = df.rename(columns={'Squad': 'Team', 'Pts': 'Points', 'GD': 'Goal_Difference'})

    # 2.3 กรองข้อมูลที่ไม่ต้องการ (เช่น แถวหัวตารางซ้ำ หรือแถวว่าง)
    df = df.dropna(subset=['Team'])
    df = df[df['Team'] != 'Squad']

    # 2.4 แปลงคอลัมน์ตัวเลขให้เป็นชนิดตัวเลข
    numeric_cols = ['Points', 'Goal_Difference']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 2.5 จัดเรียงข้อมูลตามคะแนน (Points) และผลต่างประตูได้เสีย (Goal_Difference)
    df = df.sort_values(by=['Points', 'Goal_Difference'], ascending=[False, False]).reset_index(drop=True)

    # แสดงตัวอย่างข้อมูล
    print("\nตัวอย่างข้อมูล (Top 5 Teams):")
    print(df[['Team', 'Points', 'Goal_Difference']].head())

    # --- 3. Plot Graph โดยใช้ Plotly Express ---

    if not df.empty:
        # เตรียมข้อมูลให้อยู่ในรูปแบบ 'long format' สำหรับการพลอตกราฟแท่งคู่
        plot_df = df.melt(id_vars='Team', value_vars=['Points', 'Goal_Difference'],
                          var_name='Metric', value_name='Value')

        fig = px.bar(
            plot_df,
            x='Team',
            y='Value',
            color='Metric',
            barmode='group', # จัดกลุ่มแท่งกราฟให้อยู่ด้วยกัน
            title='Premier League Team Performance: Points vs. Goal Difference',
            labels={'Team': 'ทีม', 'Value': 'ค่า', 'Metric': 'เมตริก'},
            color_discrete_map={'Points': 'blue', 'Goal_Difference': 'orange'}
        )

        # จัดเรียงแกน X ตามลำดับคะแนนในตาราง
        fig.update_layout(
            xaxis={'categoryorder':'array', 'categoryarray': df['Team'].tolist()},
            xaxis_tickangle=-45
        )
        fig.show()

        # บันทึกกราฟเป็นไฟล์ HTML (Plotly จะสร้างไฟล์ที่สามารถโต้ตอบได้)
        '''
        plot_filename = 'premier_league_performance_plot.html'
        fig.write_html(plot_filename)
        print(f"\nกราฟ Plotly ถูกบันทึกเป็นไฟล์: {plot_filename}")
        # เนื่องจากระบบไม่สามารถแสดงกราฟ Plotly ได้โดยตรง แต่คุณสามารถเปิดไฟล์ HTML นี้เพื่อดูกราฟได้
        '''


ตัวอย่างข้อมูล (Top 5 Teams):
              Team  Points  Goal_Difference
0          Arsenal      22               13
1      Bournemouth      18                5
2        Tottenham      17               10
3       Sunderland      17                4
4  Manchester City      16               10


Link : Web

In [ ]:
https://fbref.com/en/comps/9/Premier-League-Stats#all_results2025-202691

# Grok

In [ ]:
pip install requests beautifulsoup4 pandas plotly lxml

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time

# Step 1: Web Scraping
url = "https://fbref.com/en/comps/9/Premier-League-Stats" # Use the main stats page URL
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

df = None # Initialize df to None

try:
    print(f"Attempting to fetch data from: {url}")
    response = requests.get(url, headers=headers, timeout=30)
    response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
    soup = BeautifulSoup(response.content, 'lxml')

    # Attempt to find the table using multiple potential IDs or by content
    table = None
    possible_ids = [
        'results2025-202691_overall',
        'results2024-202591_overall', # Include previous season ID just in case
        'stats_squads_standard_for'
    ]

    for table_id in possible_ids:
        table = soup.find('table', id=table_id)
        if table:
            print(f"Found table with ID: {table_id}")
            break

    # Fallback: Use pandas.read_html if specific ID not found
    if not table:
        print("Specific table ID not found, attempting to use pandas.read_html with match='Squad'")
        tables = pd.read_html(response.text, match='Squad')
        if tables:
            df = tables[0].copy()
            print(f"Successfully read table with {len(df)} rows using pandas.read_html")
        else:
            print("Could not find any table matching 'Squad' with pandas.read_html")

    if table and df is None: # If table was found by ID but not yet processed by read_html fallback
        print("Processing table found by ID...")
        # If a table was found by ID, process it manually (as in the original code)
        rows = table.find('tbody').find_all('tr')
        data = []
        for row in rows:
            cells = row.find_all('td')
            if len(cells) > 0:
                # Skip header rows that might appear in tbody
                if cells[0].text.strip() in ['Rk', 'Squad']:
                    continue
                try:
                    # Adjusting cell indices based on typical FBref standings table structure
                    squad_cell = cells[0].find('a').text.strip() if cells[0].find('a') else cells[0].text.strip()
                    mp = cells[1].text.strip()
                    w = cells[2].text.strip()
                    d = cells[3].text.strip()
                    l = cells[4].text.strip()
                    gf = cells[5].text.strip()
                    ga = cells[6].text.strip()
                    gd = cells[7].text.strip()
                    pts = cells[8].text.strip()
                    # Check if xG/xGA columns exist (indices might vary)
                    xg = float(cells[9].text.strip()) if len(cells) > 9 and cells[9].text.strip() else None
                    xga = float(cells[10].text.strip()) if len(cells) > 10 and cells[10].text.strip() else None


                    data.append({
                        'Squad': squad_cell,
                        'MP': int(mp),
                        'W': int(w),
                        'D': int(d),
                        'L': int(l),
                        'GF': int(gf),
                        'GA': int(ga),
                        'GD': int(gd),
                        'Pts': int(pts),
                        'xG': xg,
                        'xGA': xga
                    })
                except (ValueError, IndexError) as cell_error:
                    print(f"Skipping row due to parsing error: {cell_error} in row {row.text[:50]}...")
                    continue # Skip this row and try the next one

        if data:
            df = pd.DataFrame(data)
            print(f"Successfully processed table data into DataFrame with {len(df)} rows.")
        else:
            print("No valid data extracted from the table.")


except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching the URL: {e}")
except Exception as e:
    print(f"❌ An unexpected error occurred during scraping or parsing: {e}")

# --- Continue with analysis and plotting only if DataFrame was created ---
if df is not None and not df.empty:
    print("\nDataFrame:")
    print(df.head(10))  # Display first 10 rows

    # Step 3: Plot Graphs with Plotly Express
    # Ensure numeric columns are correct type
    numeric_cols = ['MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'xG', 'xGA']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce') # Use coerce to turn errors into NaN

    df = df.dropna(subset=['Pts']) # Remove rows with no points data

    if not df.empty:
        # Sort for plotting
        df_sorted = df.sort_values(by='Pts', ascending=False)

        # Grap h 1: Bar chart - คะแนนทีม (Top 10)
        fig1 = px.bar(df_sorted.head(10), x='Squad', y='Pts', title='Top 10 Teams by Points (Premier League)',
                      color='Pts', color_continuous_scale='Viridis')
        fig1.update_layout(xaxis_tickangle=-45)
        fig1.show()

        # Grap h 2: Scatter plot - GF vs xG (เปรียบเทียบประสิทธิภาพการทำประตู)
        # Only plot if xG/xGA columns exist
        if 'xG' in df.columns and 'GF' in df.columns:
             fig2 = px.scatter(df, x='xG', y='GF', size='Pts', color='Squad',
                              title='Goals For (GF) vs Expected Goals (xG)',
                              hover_data=['Squad', 'Pts'])
             if df['xG'].mean() is not None:
                fig2.add_hline(y=df['xG'].mean(), line_dash="dash", line_color="red", annotation_text=f"Average xG: {df['xG'].mean():.2f}")
             if df['GF'].mean() is not None:
                 fig2.add_vline(x=df['GF'].mean(), line_dash="dash", line_color="blue", annotation_text=f"Average GF: {df['GF'].mean():.2f}")
             fig2.show()
        else:
            print("\nSkipping GF vs xG plot: xG or GF column not found.")


        # Grap h 3: Subplot - GD vs Pts และ xGD vs Pts (สำหรับ insights เพิ่มเติม)
        if 'xG' in df.columns and 'xGA' in df.columns and 'GD' in df.columns:
            # คำนวณ xGD
            df['xGD'] = df['xG'] - df['xGA']
            fig3 = make_subplots(rows=1, cols=2, subplot_titles=('Goal Difference vs Points', 'xGD vs Points'))
            fig3.add_trace(go.Scatter(x=df['GD'], y=df['Pts'], mode='markers', name='GD vs Pts', text=df['Squad']), row=1, col=1)
            fig3.add_trace(go.Scatter(x=df['xGD'], y=df['Pts'], mode='markers', name='xGD vs Pts', text=df['Squad']), row=1, col=2)
            fig3.update_layout(title_text="Performance Analysis: Actual vs Expected")
            fig3.show()
        else:
             print("\nSkipping GD vs Pts and xGD vs Pts subplot: xG, xGA, or GD column not found.")


        # Step 4: สรุป Insights (วิเคราะห์ข้อมูล)
        print("\n=== Insights จากข้อมูลผลฟุตบอล Premier League ===")

        # Check if essential columns exist before generating insights
        if all(col in df.columns for col in ['Squad', 'Pts', 'GD', 'GF', 'GA', 'xG', 'xGA', 'MP']):
            top_team = df_sorted.iloc[0]
            print(f"1. ทีมนำตาราง: {top_team['Squad']} มีคะแนนสูงสุด ({top_team['Pts']} Pts จาก {top_team['MP']} นัด) ด้วย GD {top_team['GD']}")

            # Find team with largest xG_diff (overperforming offense)
            df['xG_diff'] = df['GF'] - df['xG']
            overperforming_offense = df.nlargest(1, 'xG_diff').iloc[0]
            print(f"2. Overperforming Offense (GF > xG): {overperforming_offense['Squad']} (GF={overperforming_offense['GF']}, xG={overperforming_offense['xG']:.2f})")

            # Find team with largest negative xG_diff (underperforming offense)
            underperforming_offense = df.nsmallest(1, 'xG_diff').iloc[0]
            print(f"3. Underperforming Offense (GF < xG): {underperforming_offense['Squad']} (GF={underperforming_offense['GF']}, xG={underperforming_offense['xG']:.2f})")


            # Find team with smallest xGA_diff (overperforming defense - conceding less than expected)
            df['xGA_diff'] = df['GA'] - df['xGA']
            overperforming_defense = df.nsmallest(1, 'xGA_diff').iloc[0]
            print(f"4. Overperforming Defense (GA < xGA): {overperforming_defense['Squad']} (GA={overperforming_defense['GA']}, xGA={overperforming_defense['xGA']:.2f})")

             # Find team with largest xGA_diff (underperforming defense - conceding more than expected)
            underperforming_defense = df.nlargest(1, 'xGA_diff').iloc[0]
            print(f"5. Underperforming Defense (GA > xGA): {underperforming_defense['Squad']} (GA={underperforming_defense['GA']}, xGA={underperforming_defense['xGA']:.2f})")


            print(f"สถิติรวม: Average Pts = {df['Pts'].mean():.2f}, Average GF = {df['GF'].mean():.2f}, Average xG = {df['xG'].mean():.2f}")

        else:
            print("Cannot generate detailed insights due to missing columns.")

    else:
        print("\nDataFrame is empty after cleaning. Cannot generate plots or insights.")

else:
    print("\nCould not create DataFrame from web scraping or it is empty.")

Attempting to fetch data from: https://fbref.com/en/comps/9/Premier-League-Stats
❌ Error fetching the URL: 403 Client Error: Forbidden for url: https://fbref.com/en/comps/9/Premier-League-Stats

Could not create DataFrame from web scraping or it is empty.


In [ ]:
pip install cloudscraper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 4.8 MB/s eta 0:00:00


# Web Scrape data

In [ ]:
# Simple web scraping : NOT work

import pandas as pd

df = pd.read_html('https://fbref.com/en/comps/8/Premier-League-Stats', attrs={"id":"results2025-202691_overall"})[0]

#results2025-202691_overall

ValueError: No tables found

In [ ]:
import cloudscraper

# 1. สร้าง Cloudscraper instance
# scraper จะทำหน้าที่จัดการการส่งคำขอ HTTP เหมือนกับ requests.Session()
scraper = cloudscraper.create_scraper()

# 2. กำหนด URL เป้าหมาย (ในตัวอย่างนี้ใช้ URL ที่สมมติขึ้น)
# คุณสามารถลองใช้กับเว็บไซต์ที่ทราบว่ามีการป้องกันของ Cloudflare
#url = "https://example.com"

url ="https://fbref.com/en/comps/8/Premier-League-Stats"

# 3. ส่งคำขอ GET ไปยัง URL
response = scraper.get(url)

# 4. ตรวจสอบสถานะการตอบกลับและแสดงผลเนื้อหา
if response.status_code == 200:
    print("การดึงข้อมูลสำเร็จ! สถานะ:", response.status_code)
    # แสดงเนื้อหา HTML บางส่วนที่ได้รับกลับมา
    print("\nตัวอย่างเนื้อหา (500 ตัวอักษรแรก):\n")
    print(response.text[:500])
else:
    print(f"การดึงข้อมูลล้มเหลว สถานะ: {response.status_code}")

การดึงข้อมูลล้มเหลว สถานะ: 403


In [ ]:
import cloudscraper
import pandas as pd
import time
from io import StringIO

# 1. กำหนด URL และ Table ID
URL = "https://fbref.com/en/comps/9/Premier-League-Stats#all_results2025-202691"
TABLE_ID = "results2025-202691" # ID ของตาราง Scores & Fixtures

# 2. สร้าง Cloudscraper Instance
# Cloudscraper จะทำหน้าที่จัดการการ Bypass การตรวจสอบของ Cloudflare
scraper = cloudscraper.create_scraper()

# 3. ส่งคำขอ GET เพื่อดึงเนื้อหา HTML
print("กำลังดึงข้อมูลจาก FBref...")
try:
    response = scraper.get(URL)
    # เพิ่มการหน่วงเวลาเพื่อหลีกเลี่ยงการถูกจำกัด (Rate Limit)
    time.sleep(5)

    # 4. ลบคอมเมนต์ HTML ออก
    # ตารางใน FBref ส่วนใหญ่ถูกห่อหุ้มด้วย ซึ่ง Pandas/BeautifulSoup จะมองไม่เห็น
    # เราจึงแทนที่ ด้วยข้อความว่างเปล่า
    html_content = response.text.replace('', '')

    # 5. ใช้ Pandas เพื่ออ่านตาราง HTML
    # เราใช้ attrs={'id': TABLE_ID} เพื่อระบุตารางที่ต้องการ
    # StringIO(html_content) ใช้เพื่อให้ pd.read_html อ่านจาก String ได้เหมือนไฟล์
    dataframes = pd.read_html(StringIO(html_content), attrs={'id': TABLE_ID})

    # ตารางที่ได้จะอยู่ในรูปแบบ List ของ DataFrame
    df = dataframes[0]

    # 6. ทำความสะอาด DataFrame (เฉพาะตัวอย่าง)
    # FBref มักจะมี Column Header สองชั้น เราสามารถ Drop ชั้นบน (level 0) ออกได้
    # หรือจะรวมชื่อ Column เข้าด้วยกันก็ได้

    # *ตัวอย่างการลบ Column Header ชั้นบน (หากมี)
    # df.columns = df.columns.droplevel()

    # *ตัวอย่างการเปลี่ยนชื่อ Column เพื่อความชัดเจน
    df.columns = [f"{col[1]}" for col in df.columns]

    # แสดงผลข้อมูลที่ดึงมา
    print("\n✅ ดึงข้อมูลสำเร็จ! แสดงผล 5 แถวแรก:")
    print(df.head())
    print("\nขนาดของ DataFrame:", df.shape)

except Exception as e:
    print(f"\n❌ เกิดข้อผิดพลาดในการดึงข้อมูล: {e}")
    if response.status_code != 200:
        print(f"รหัสสถานะที่ได้รับ: {response.status_code}. อาจถูกบล็อก.")




กำลังดึงข้อมูลจาก FBref...

❌ เกิดข้อผิดพลาดในการดึงข้อมูล: No tables found
รหัสสถานะที่ได้รับ: 403. อาจถูกบล็อก.


In [ ]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 28.6 MB/s eta 0:00:00


/tmp/ipython-input-3859580330.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(data, attrs={"id":"results2025-202691_overall"})


In [5]:
df

[    Rk            Squad  MP  W  D  L  GF  GA  GD  Pts  Pts/MP    xG   xGA  \
 0    1          Arsenal   9  7  1  1  16   3  13   22    2.44  14.6   5.2   
 1    2      Bournemouth   9  5  3  1  16  11   5   18    2.00  11.0  11.1   
 2    3        Tottenham   9  5  2  2  17   7  10   17    1.89   9.9  10.7   
 3    4       Sunderland   9  5  2  2  11   7   4   17    1.89   8.6  10.4   
 4    5  Manchester City   9  5  1  3  17   7  10   16    1.78  15.6   9.0   
 5    6   Manchester Utd   9  5  1  3  15  14   1   16    1.78  16.2  13.6   
 6    7        Liverpool   9  5  0  4  16  14   2   15    1.67  15.8  12.0   
 7    8      Aston Villa   9  4  3  2   9   8   1   15    1.67   7.1  11.8   
 8    9          Chelsea   9  4  2  3  17  11   6   14    1.56  13.4  13.1   
 9   10   Crystal Palace   9  3  4  2  12   9   3   13    1.44  17.6  11.7   
 10  11        Brentford   9  4  1  4  14  14   0   13    1.44  13.4  12.2   
 11  12    Newcastle Utd   9  3  3  3   9   8   1   12    1.33  

In [ ]:
#!pip install selenium

from selenium import webdriver
import pandas as pd
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    # options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()
data=[]
#driver.get('https://fbref.com/en/comps/9/Premier-League-Stats#all_results2025-202691')
driver.get('https://fbref.com/en/comps/9/Premier-League-Stats')
data = driver.page_source

df = pd.read_html(data, attrs={"id":"results2025-202691_overall"})
df

/tmp/ipython-input-2679361105.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(data, attrs={"id":"results2025-202691_overall"})


[    Rk            Squad  MP  W  D  L  GF  GA  GD  Pts  Pts/MP    xG   xGA  \
 0    1          Arsenal   9  7  1  1  16   3  13   22    2.44  14.6   5.2   
 1    2      Bournemouth   9  5  3  1  16  11   5   18    2.00  11.0  11.1   
 2    3        Tottenham   9  5  2  2  17   7  10   17    1.89   9.9  10.7   
 3    4       Sunderland   9  5  2  2  11   7   4   17    1.89   8.6  10.4   
 4    5  Manchester City   9  5  1  3  17   7  10   16    1.78  15.6   9.0   
 5    6   Manchester Utd   9  5  1  3  15  14   1   16    1.78  16.2  13.6   
 6    7        Liverpool   9  5  0  4  16  14   2   15    1.67  15.8  12.0   
 7    8      Aston Villa   9  4  3  2   9   8   1   15    1.67   7.1  11.8   
 8    9          Chelsea   9  4  2  3  17  11   6   14    1.56  13.4  13.1   
 9   10   Crystal Palace   9  3  4  2  12   9   3   13    1.44  17.6  11.7   
 10  11        Brentford   9  4  1  4  14  14   0   13    1.44  13.4  12.2   
 11  12    Newcastle Utd   9  3  3  3   9   8   1   12    1.33  

In [ ]:
df

[    Rk            Squad  MP  W  D  L  GF  GA  GD  Pts  Pts/MP    xG   xGA  \
 0    1          Arsenal   9  7  1  1  16   3  13   22    2.44  14.6   5.2   
 1    2      Bournemouth   9  5  3  1  16  11   5   18    2.00  11.0  11.1   
 2    3        Tottenham   9  5  2  2  17   7  10   17    1.89   9.9  10.7   
 3    4       Sunderland   9  5  2  2  11   7   4   17    1.89   8.6  10.4   
 4    5  Manchester City   9  5  1  3  17   7  10   16    1.78  15.6   9.0   
 5    6   Manchester Utd   9  5  1  3  15  14   1   16    1.78  16.2  13.6   
 6    7        Liverpool   9  5  0  4  16  14   2   15    1.67  15.8  12.0   
 7    8      Aston Villa   9  4  3  2   9   8   1   15    1.67   7.1  11.8   
 8    9          Chelsea   9  4  2  3  17  11   6   14    1.56  13.4  13.1   
 9   10   Crystal Palace   9  3  4  2  12   9   3   13    1.44  17.6  11.7   
 10  11        Brentford   9  4  1  4  14  14   0   13    1.44  13.4  12.2   
 11  12    Newcastle Utd   9  3  3  3   9   8   1   12    1.33  